# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [23]:
import requests
import pyowm
import json
from getpass import getpass

api_key = getpass("Enter your API Key: ").strip()

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [24]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 285.97,
  'feels_like': 285.63,
  'temp_min': 284.74,
  'temp_max': 287.03,
  'pressure': 1032,
  'humidity': 89,
  'sea_level': 1032,
  'grnd_level': 1028},
 'visibility': 10000,
 'wind': {'speed': 1.34, 'deg': 241, 'gust': 3.13},
 'clouds': {'all': 100},
 'dt': 1729674037,
 'sys': {'type': 2,
  'id': 2091269,
  'country': 'GB',
  'sunrise': 1729665516,
  'sunset': 1729702241},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [25]:
#your code here

from pyowm.owm import OWM
owm = OWM(api_key)
mgr = owm.weather_manager()

observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather
weather.status           # short version of status (eg. 'Rain')
weather.detailed_status  # detailed version of status (eg. 'light rain')

'overcast clouds'

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [26]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 285.97°C, Humidity: 89%, Wind Speed: 1.34 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [27]:
#your code here

observation = mgr.weather_at_place('London,GB')
weather = observation.weather

temperature = weather.temperature('celsius')['temp']
humidity = weather.humidity  
wind_speed = weather.wind()['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")


Temperature: 12.82°C, Humidity: 89%, Wind Speed: 1.34 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [28]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [29]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2024-10-23 12:00:00, Temp: 288.78°C
Time: 2024-10-23 15:00:00, Temp: 291.48°C
Time: 2024-10-23 18:00:00, Temp: 296.66°C
Time: 2024-10-23 21:00:00, Temp: 296.87°C
Time: 2024-10-24 00:00:00, Temp: 293.6°C
Time: 2024-10-24 03:00:00, Temp: 292.01°C
Time: 2024-10-24 06:00:00, Temp: 291.48°C
Time: 2024-10-24 09:00:00, Temp: 290.06°C
Time: 2024-10-24 12:00:00, Temp: 287.28°C
Time: 2024-10-24 15:00:00, Temp: 288.52°C
Time: 2024-10-24 18:00:00, Temp: 291.34°C
Time: 2024-10-24 21:00:00, Temp: 291.41°C
Time: 2024-10-25 00:00:00, Temp: 288.64°C
Time: 2024-10-25 03:00:00, Temp: 286.02°C
Time: 2024-10-25 06:00:00, Temp: 284.88°C
Time: 2024-10-25 09:00:00, Temp: 283.69°C
Time: 2024-10-25 12:00:00, Temp: 283.24°C
Time: 2024-10-25 15:00:00, Temp: 286.2°C
Time: 2024-10-25 18:00:00, Temp: 288.42°C
Time: 2024-10-25 21:00:00, Temp: 288.41°C
Time: 2024-10-26 00:00:00, Temp: 288.42°C
Time: 2024-10-26 03:00:00, Temp: 287.77°C
Time: 2024-10-26 06:00:00, Temp: 287.44°C
Time: 2024-10-26 09:00:00, Temp: 287

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [30]:
#your code here

forecast = mgr.forecast_at_place('New York,US', '3h')

for weather in forecast.forecast:
    time = weather.reference_time('iso')
    detailed_status = weather.detailed_status
    temperature = weather.temperature('celsius')['temp']
    
    print(f"Time: {time}, Status: {detailed_status}, Temperature: {temperature}°C")


Time: 2024-10-23 12:00:00+00:00, Status: clear sky, Temperature: 15.53°C
Time: 2024-10-23 15:00:00+00:00, Status: scattered clouds, Temperature: 18.28°C
Time: 2024-10-23 18:00:00+00:00, Status: few clouds, Temperature: 23.51°C
Time: 2024-10-23 21:00:00+00:00, Status: clear sky, Temperature: 23.72°C
Time: 2024-10-24 00:00:00+00:00, Status: clear sky, Temperature: 20.45°C
Time: 2024-10-24 03:00:00+00:00, Status: clear sky, Temperature: 18.86°C
Time: 2024-10-24 06:00:00+00:00, Status: clear sky, Temperature: 18.33°C
Time: 2024-10-24 09:00:00+00:00, Status: broken clouds, Temperature: 16.91°C
Time: 2024-10-24 12:00:00+00:00, Status: broken clouds, Temperature: 14.13°C
Time: 2024-10-24 15:00:00+00:00, Status: scattered clouds, Temperature: 15.37°C
Time: 2024-10-24 18:00:00+00:00, Status: few clouds, Temperature: 18.19°C
Time: 2024-10-24 21:00:00+00:00, Status: clear sky, Temperature: 18.26°C
Time: 2024-10-25 00:00:00+00:00, Status: clear sky, Temperature: 15.49°C
Time: 2024-10-25 03:00:00+0

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [31]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 285.97°C, Fahrenheit: 546.7460000000001°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [32]:
#your code here

observation = mgr.weather_at_place('London,GB')
weather = observation.weather

temperature_celsius = weather.temperature('celsius')['temp']
temperature_fahrenheit = weather.temperature('fahrenheit')['temp']

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")


Temperature in Celsius: 12.89°C, Fahrenheit: 55.2°F
